In [1]:
import pandas as pd
from textaugment import EDA
import nltk
import transformers

FileNotFoundError: [Errno 2] No such file or directory: 'Yelpdata.csv'

In [ ]:
!pip install textaugment
!pip install transformers
nltk.download('wordnet')
nltk.download('stopwords')

In [2]:
data=pd.read_json("Yelpdata.json")
pd.options.mode.chained_assignment = None 
data

,Rating,Review
0,1,I got 'new' tires from them and within two wee...
1,1,Don't waste your time. We had two different p...
2,1,All I can say is the worst! We were the only 2...
3,1,I have been to this restaurant twice and was d...
4,1,Food was NOT GOOD at all! My husband & I ate h...
...,...,...
194,5,Had the best burger of my life at Tessaro's ov...
195,5,Friendly Service. Fresh Veggies! At only 1 dol...
196,5,"I only come to Pittsburgh twice a year, but I ..."
197,5,Yes the pizza is the best in the burgh. But d...


In [4]:
eda=EDA()

#Random insertion
for i in range(5):
  new_review=eda.random_insertion(data['Review'][i])
  edit=pd.DataFrame({'Rating':data['Rating'][i],'Review':[new_review]})
  data=data.append(edit,ignore_index = True)


#Random deletion
for i in range(70,75):
  new_review=eda.random_deletion(data['Review'][i],p=0.3)
  edit=pd.DataFrame({'Rating':data['Rating'][i],'Review':[new_review]})
  data=data.append(edit,ignore_index = True)

#Synonym Replacement
for i in range(100,105):
  new_review=eda.synonym_replacement(data['Review'][i])
  edit=pd.DataFrame({'Rating':data['Rating'][i],'Review':[new_review]})
  data=data.append(edit,ignore_index = True)

#Random Swap
for i in range(180,185):
  new_review=eda.random_swap(data['Review'][i])
  edit=pd.DataFrame({'Rating':data['Rating'][i],'Review':[new_review]})
  data=data.append(edit,ignore_index = True)


In [5]:
EDA_augumentation() 
data

,Rating,Review
0,1,I got 'new' tires from them and within two wee...
1,1,Don't waste your time. We had two different p...
2,1,All I can say is the worst! We were the only 2...
3,1,I have been to this restaurant twice and was d...
4,1,Food was NOT GOOD at all! My husband & I ate h...
...,...,...
214,4,We stayed here after getting an awesome deal o...
215,4,This restaurant is great for anyone in the moo...
216,4,I really really really love Harp & Fiddle. Tho...
217,5,Haven't been in a while (not really a bike nut...


In [6]:
c=0
for i in range(len(data)):
  if(data['Rating'][i]>=4):
    #print(i)
    data['Rating'][i]=1
  else:
    data['Rating'][i]=0

In [7]:
data['Rating'].value_counts()

1    111
0    108
Name: Rating, dtype: int64

In [9]:
ModelClass=transformers.DistilBertModel
TokenizerClass=transformers.DistilBertTokenizer

model=ModelClass.from_pretrained('distilbert-base-uncased') 
tokenizer=TokenizerClass.from_pretrained('distilbert-base-uncased')

##Data Preprocessing

Tokenization  
Reviews must be tokenized and embedded in order to provide it to the DistilBert model

In [10]:
tokenized_reviews= data['Review'].apply((lambda review: tokenizer.encode(review, add_special_tokens=True, truncation=True)))
temp=tokenized_reviews

In [11]:
tokenized_reviews
#Each review is represented as a list of tokens

0      [101, 1045, 2288, 1005, 2047, 1005, 13310, 201...
1      [101, 2123, 1005, 1056, 5949, 2115, 2051, 1012...
2      [101, 2035, 1045, 2064, 2360, 2003, 1996, 5409...
3      [101, 1045, 2031, 2042, 2000, 2023, 4825, 3807...
4      [101, 2833, 2001, 2025, 2204, 2012, 2035, 999,...
                             ...                        
214    [101, 2057, 4370, 2182, 2044, 2893, 2019, 1247...
215    [101, 2023, 4825, 2003, 2307, 2005, 3087, 1999...
216    [101, 1045, 2428, 2428, 2428, 2293, 14601, 100...
217    [101, 4033, 1005, 1056, 2042, 1999, 1037, 2096...
218    [101, 2070, 7144, 3722, 3062, 2046, 2026, 2067...
Name: Review, Length: 219, dtype: object

In [12]:
max_num=0

for tokens in tokenized_reviews:
  max_num=max(max_num,len(tokens))

In [13]:
for index in range(len(tokenized_reviews)):
  padding_needed=max_num-len(tokenized_reviews[index])
  tokenized_reviews[index]=tokenized_reviews[index]+[0]*(padding_needed)

In [14]:
import numpy as np
padded_reviews=np.array([tokens for tokens in tokenized_reviews.values])
padded_reviews

array([[ 101, 1045, 2288, ...,    0,    0,    0],
       [ 101, 2123, 1005, ...,    0,    0,    0],
       [ 101, 2035, 1045, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2428, ...,    0,    0,    0],
       [ 101, 4033, 1005, ...,    0,    0,    0],
       [ 101, 2070, 7144, ...,    0,    0,    0]])

In [15]:
#if elements of padded_reviews does not contain 0, then element=1, otherwise element=0
attentionmask=np.where(padded_reviews != 0,1,0)

In [16]:
import torch 
inputIDs=torch.tensor(padded_reviews) #input to model
attentionmask=torch.tensor(attentionmask) #to ignore the padding

with torch.no_grad():
      Output_3D = model(inputIDs, attention_mask=attentionmask)

In [17]:
feature_matrix = Output_3D[0][:,0,:].numpy()
Sentiment=data['Rating'] #Class

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feature_matrix,Sentiment)

In [19]:
from sklearn import naive_bayes
nb=naive_bayes.BernoulliNB()
nb.fit(x_train,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [20]:
from sklearn.metrics import accuracy_score
prediction=nb.predict(x_test)
accuracy_score(prediction,y_test)

0.8